# ETL and Data Cleanup Project

This notebook performs an end-to-end ETL and data cleaning pipeline on a messy customer sales dataset.  
It covers data loading, exploration, cleaning of missing or inconsistent data, handling malformed dates, and exporting the cleaned dataset to CSV and SQLite.

In [3]:
import pandas as pd
import sqlite3
from datetime import datetime

# Load the raw messy sales data CSV
sales = pd.read_csv('messy_customer_sales.csv')

# Display the first few rows
sales.head()

,customer_id,customer_name,email,signup_date,last_purchase_date,purchase_amount,product_category,country
0,145.0,Ryan Perez,ryan.perez@example.com,04-29-2025,21/03/2025,133.82,Fashion,france
1,328.0,Rachel Donovan,NaN,07/08/2024,09-11-2024,210.23,Electronics,France
2,358.0,CHRISTOPHER HARRIS,christopher.harris@example.com,13/06/2025,2024-08-24,288.82,Electronics,uk
3,435.0,Patricia Caldwell,patricia.caldwell@example.com,17/06/2025,15/02/2025,312.42,Fashion,UK
4,429.0,Lori Nelson,lorinelson@@example.com,2024-10-30,08-25-2024,159.17,Fashion,france


## Initial Data Exploration

We start by examining the structure of the dataset.  
`info()` shows the number of non-null values per column and data types, which helps identify missing data and columns requiring type conversion.  
`describe()` provides summary statistics for numerical columns.

In [4]:
# Basic overview of data types and completeness
sales.info()

# Summary statistics for numeric columns
sales.describe()

# Show counts of unique product categories and countries to check consistency
print("Product categories:\n", sales['product_category'].value_counts())
print("Countries:\n", sales['country'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         981 non-null    float64
 1   customer_name       1000 non-null   object 
 2   email               955 non-null    object 
 3   signup_date         1000 non-null   object 
 4   last_purchase_date  899 non-null    object 
 5   purchase_amount     1000 non-null   float64
 6   product_category    1000 non-null   object 
 7   country             949 non-null    object 
dtypes: float64(2), object(6)
memory usage: 62.6+ KB
Product categories:
 product_category
Home           290
Fashion        286
Electronics    278
electronics     34
home            27
fashion         27
Hom             20
Fashio          20
Electronic      15
fashio           2
hom              1
Name: count, dtype: int64
Countries:
 country
uk                114
UK                114
United States  

## Cleaning Customer IDs

The `customer_id` column contains missing and malformed entries.  
We convert it to string type to identify missing values, then assign new unique IDs to missing entries, and finally convert back to a nullable integer type.

In [6]:
# Convert to string for cleaning
sales['customer_id'] = sales['customer_id'].astype(str)

# Identify missing IDs (strings 'nan', empty or whitespace)
missing_mask = sales['customer_id'].isin(['nan', '', ' '])

# Extract existing valid numeric IDs
existing_ids = pd.to_numeric(sales.loc[~missing_mask, 'customer_id'], errors='coerce').dropna().astype(int)

max_id = existing_ids.max() if not existing_ids.empty else 0

# Assign new zero-padded IDs for missing entries
new_ids = range(max_id + 1, max_id + 1 + missing_mask.sum())
sales.loc[missing_mask, 'customer_id'] = [f'{i:03d}' for i in new_ids]

# Ensure consistent zero-padding
sales['customer_id'] = sales['customer_id'].astype(str).str.zfill(3)

# Convert to nullable integer type
sales['customer_id'] = pd.to_numeric(sales['customer_id'], errors='coerce').astype('Int64')

print("Cleaned customer_id dtype:", sales['customer_id'].dtype)

Cleaned customer_id dtype: Int64


## Cleaning Product Categories

The `product_category` column has inconsistent spelling and case.  
We standardize by converting to lowercase and fixing common typos.

sales['product_category'] = sales['product_category'].str.lower().replace({
    'hom': 'home',
    'electronic': 'electronics',
    'fashio': 'fashion'
})

print(sales['product_category'].value_counts())

## Cleaning Country Names

Similarly, country names vary in format.  
We normalize to lowercase and standardize common names.

In [8]:
sales['country'] = sales['country'].str.lower().replace({
    'united states': 'usa',
    'united kingdom': 'uk',
    'fra': 'france'
})

print(sales['country'].value_counts())

country
uk        329
usa       312
france    308
Name: count, dtype: int64


## Cleaning Emails

Emails often contain extra spaces or typos like double '@'.  
We strip whitespace, convert to lowercase, and fix known issues.

In [9]:
sales['email'] = sales['email'].str.strip().str.lower().str.replace('@@', '@', regex=False)

print("Missing emails:", sales['email'].isna().sum())
print("Top email addresses:\n", sales['email'].value_counts().head(20))

Missing emails: 45
Top email addresses:
 email
andrew.johnson@example.com     2
david.clark@example.com        2
jeffrey.clark@example.com      2
joseph.santos@example.com      2
mark.adams@example.com         2
chase.smith@example.com        2
rebecca.miller@example.com     2
samantha.moore@example.com     2
andrea.hart@example.com        2
pamela.bell@example.com        1
barbara.flores@example.com     1
edward.herman@example.com      1
deanna.ochoa@example.com       1
kelly.williams@example.com     1
thomas.martinez@example.com    1
mark.golden@example.com        1
joseph.rivera@example.com      1
peter.hall@example.com         1
william.smith@example.com      1
brenda.white@example.com       1
Name: count, dtype: int64


## Handling Purchase Amounts

Negative purchase amounts are ambiguous (could be returns or errors).  
To maintain data integrity, we remove rows with negative purchase amounts.

In [13]:
print(f"Number of negative purchase amounts: {(sales['purchase_amount'] < 0).sum()}")

sales = sales[sales['purchase_amount'] >= 0]

Number of negative purchase amounts: 0


## Parsing and Standardizing Dates

The `signup_date` and `last_purchase_date` columns contain dates in mixed formats.  
We define common date formats and apply a parser that tries each format until one succeeds, converting to Python date objects without time components.

In [14]:
from datetime import datetime
import pandas as pd

common_formats = [
    '%Y-%m-%d',     # 2023-12-31
    '%d-%m-%Y',     # 31-12-2023
    '%m/%d/%Y',     # 12/31/2023
    '%d.%m.%Y',     # 31.12.2023
    '%b %d, %Y',    # Dec 31, 2023
    '%B %d, %Y',    # December 31, 2023
    '%Y/%m/%d',     # 2023/12/31
]

def try_parse_date(date_str):
    if pd.isna(date_str):
        return pd.NaT
    for fmt in common_formats:
        try:
            return datetime.strptime(date_str.strip(), fmt)
        except (ValueError, TypeError):
            continue
    return pd.NaT

for col in ['signup_date', 'last_purchase_date']:
    sales[col] = sales[col].apply(try_parse_date)  # parse with custom function
    sales[col] = pd.to_datetime(sales[col])        # convert to pandas datetime64
    sales[col] = sales[col].dt.normalize()         # zero out time (keep datetime dtype)

<class 'pandas.core.frame.DataFrame'>
Index: 971 entries, 0 to 999
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   signup_date         650 non-null    object
 1   last_purchase_date  592 non-null    object
dtypes: object(2)
memory usage: 22.8+ KB


## Exporting the Cleaned Data

We export the cleaned DataFrame both as a CSV backup and into a SQLite database for further analysis.

In [15]:
# Export to CSV
sales.to_csv('cleaned_sales_backup.csv', index=False)

# Export to SQLite database
conn = sqlite3.connect('cleaned_sales.db')
sales.to_sql('sales_data', conn, if_exists='replace', index=False)
conn.close()

## Verification of Export

To verify the SQLite export, we read back a few rows from the database.

In [16]:
conn = sqlite3.connect('cleaned_sales.db')
print(pd.read_sql("SELECT * FROM sales_data LIMIT 5;", conn))
conn.close()

   customer_id       customer_name                           email  \
0          145          Ryan Perez          ryan.perez@example.com   
1          328     Rachel Donovan                             None   
2          358  CHRISTOPHER HARRIS  christopher.harris@example.com   
3          435   Patricia Caldwell   patricia.caldwell@example.com   
4          429         Lori Nelson          lorinelson@example.com   

  signup_date last_purchase_date  purchase_amount product_category country  
0        None               None           133.82          fashion  france  
1  2024-07-08         2024-11-09           210.23      electronics  france  
2        None         2024-08-24           288.82      electronics      uk  
3        None               None           312.42          fashion      uk  
4  2024-10-30               None           159.17          fashion  france  


In [17]:
sales.dtypes

customer_id             Int64
customer_name          object
email                  object
signup_date            object
last_purchase_date     object
purchase_amount       float64
product_category       object
country                object
dtype: object